**Perfect number example**

A perfect number is one where the factors of a number other than itself add up to the number.
Factors of 6 are 1,2,3,6 and 1 + 2 + 3 = 6

What do we need to test for:
- Do we get the correct response for a few known perfect numbers?
- Do we get the correct response for a few known non-perfect numbers?
- Did we factorize correctly?


In [76]:
class Numbers:

    def __init__(self):
        pass

    def perfect_number(self,n):
        factors = []
        factors.append(1)
        factors.append(n)
        for i in range(2,n):
            if (n % i == 0):
                factors.append(i)
        # sum factors
        sum = 0
        for f in factors:
                sum += f

        # decide if it's perfect
        return (sum - n == n)

In [77]:
def test_perfect_number():
    num = Numbers()
    assert num.perfect_number(6) == True
    assert num.perfect_number(28) == True
    assert num.perfect_number(30) == False

This works, but if we wanted to test every edge case this is going to start to look nasty. Instead, we can use a package called `pytest` which simplifies things. Run the line below to see the output, and then go and check what it looks like in the repository.

In [78]:
!py.test tests

============================= test session starts ==============================
platform darwin -- Python 3.5.2, pytest-3.0.5, py-1.4.32, pluggy-0.4.0
rootdir: /Users/BenRegner/dev/insight/swe-barebones, inifile: 
collected 5 items 

tests/test_numbers.py .....

=========================== 5 passed in 0.78 seconds ===========================


Lets take a look at whats going on in there using the %load magic function

In [ ]:
%load tests/test_numbers.py

The first test uses parametrize, which gives you a lot of flexibility. A great write up of everything you can do with parametrize can be found in the official documentation here: http://doc.pytest.org/en/latest/example/parametrize.html

In the second test we run a loop to test many numbers. We could have also used parametrize for this test, although each test in parametrize stands alone, so the output becomes a little long. Go to the repository (not the cell above) and try changing this example to use the parametrize decorator. Run ```$ py.test tests``` and see how the output changes.

This isn't super slow, but looking at our code we can find an inefficiency in our implementation. If we think a bit, we can see that we don't need to loop over all numbers up to the number we are checking, but can pick off pairs of factors as we find them. This means we only need to go up to the square root of the number we are checking. Below is a new implementation:

In [80]:
import numpy as np

# Don't forget self when putting into a class!
def perfect_number_optimized(n):
    factors = []
    factors.append(1)
    factors.append(n)
    for i in range(2,int(np.sqrt(n))+1):
        if (n % i == 0):
            factors.append(i)
            factors.append(n // i)
    # sum factors
    sum = 0
    print(factors)
    for f in factors:
        sum += f

    # decide if it's perfect
    return (sum - n == n)

Try putting this into the Numbers class, change the tests to use this version, and run them again. It runs a little faster, so we're making progress! But we still haven't implemented all the tests we talked about early on. Lets refactor this code to do the factorization separately, which conveniently also lets us test that we're doing factorization correctly!

In [81]:
# Don't forget self when putting into a class!
def factorize(n):
    factors = []
    factors.append(1)
    factors.append(n)
    for i in range(2,int(np.sqrt(n))+1):
        if (n % i == 0):
            factors.append(i)
            factors.append(n // i)
    return factors

def perfect_number_optimized(n):
    factors = factorize(n)
    sum = 0
    for f in factors:
        sum += f
    # decide if it's perfect
    return (sum - n == n)

In [82]:
factorize(16)

[1, 16, 2, 8, 4, 4]

Whats going on here? If we look back at our implementation, we immediately see the problem. For perfect squares, we end up adding them twice in the lines
```
factors.append(i)
factors.append(n // i)
```
There are a few ways to fix this, the easiest being a conditional check:
```
for i in range(2,int(np.sqrt(n))+1):
    if (n % i == 0):
        factors.append(i)
        if (n // i !=  i):
            factors.append(n // i)

```

Make this change in the class and try running the tests again. Now we're passing all the tests. Note that we would have missed this if we hadn't included a specific example that caught the problem. **Unit testing doesn't prevent all bugs!** But they will help you avoid many many problems.

**Github Excercise**

Build a function or set of functions to calculate the total number of distinct contributors on all repositories for any given username

API endpoints: (test it out with curl!)
List of all repos for a given username: https://api.github.com/users/{username}/repos
List of all contributors for a given username and repository: https://api.github.com/repos/{username}/{repo}/contributors 

Below you'll find some code to help get you started. How would you write tests to make sure you detect it if Github changes their API? What other tests should you write to make sure the user of your new function always gets what the expect?

In [83]:
import requests

API_ENDPOINT = "https://api.github.com"
def fetch_repo_names(userid):
    resp = requests.get("{}/users/{}/repos".format(API_ENDPOINT, userid))
    return [x['name'] for x in resp.json()]

def fetch_repo_contributors(userid, reponame):
    resp = requests.get("{}/repos/{}/{}/contributors".format(API_ENDPOINT, userid, reponame))
    return [x['login'] for x in resp.json()]